<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
##### Begin Working Here #####

Replicate the lesson code.

See my Github account:  https://github.com/ilyanovak/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features.ipynb

Convert the `term` column from string to integer.

In [1]:
! wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-01-20 02:57:16--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [          <=>       ]  21.72M   426KB/s    in 54s     

2020-01-20 02:58:11 (412 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22776791]



In [2]:
! unzip LoanStats_2018Q4.csv

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [0]:
import pandas as pd

In [58]:
df = pd.read_csv('LoanStats_2018Q4.csv', header=1, skipfooter=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [54]:
df['term'].value_counts(dropna=False)

36    88171
60    40226
Name: term, dtype: int64

In [0]:
def conv_int(months):
  return int(months.strip(' months'))

In [17]:
conv_int('36 months')

36

In [18]:
type(conv_int('36 months'))

int

In [19]:
df['term'] = df['term'].apply(conv_int)
df['term'].value_counts()

36    88171
60    40226
Name: term, dtype: int64

In [22]:
df['term'].head()

0    36
1    36
2    36
3    36
4    60
Name: term, dtype: int64

Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.

In [55]:
df['loan_status'].value_counts(dropna=False)

Current               97694
Fully Paid            21476
Charged Off            5260
Late (31-120 days)     2656
In Grace Period         784
Late (16-30 days)       524
Default                   3
Name: loan_status, dtype: int64

In [0]:
def loan_status(status):
  if (status == 'Current') or (status == 'Fully Paid'):
    return 1
  else:
    return 0

In [26]:
examples = ['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)']

[loan_status(status) for status in examples]

[1, 1, 0, 0]

In [34]:
df['loan_status_is_great'] = df['loan_status'].apply(loan_status)
df[['loan_status', 'loan_status_is_great']].head(5)

,loan_status,loan_status_is_great
0,Charged Off,0
1,Current,1
2,Fully Paid,1
3,Current,1
4,Current,1


Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.



In [35]:
for col in df.columns:
  if col.endswith('_d') or col.endswith('_date'):
    print(col)

issue_d
last_pymnt_d
next_pymnt_d
last_credit_pull_d
hardship_start_date
hardship_end_date
payment_plan_start_date
debt_settlement_flag_date
settlement_date


In [59]:
df['last_pymnt_d'].value_counts(dropna=False)

Dec-2019    98862
Nov-2019     3241
Jul-2019     3143
Oct-2019     2833
Sep-2019     2803
Aug-2019     2731
May-2019     2547
Apr-2019     2356
Jun-2019     2289
Mar-2019     1984
Feb-2019     1915
Jan-2019     1561
Dec-2018     1054
Nov-2018      679
Oct-2018      234
NaN           159
Jan-2020        6
Name: last_pymnt_d, dtype: int64

In [42]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], infer_datetime_format=True)
df['last_pymnt_d'].head(10)

0   2019-04-01
1   2019-12-01
2   2019-02-01
3   2019-12-01
4   2019-12-01
5   2019-12-01
6   2019-12-01
7   2019-12-01
8   2019-12-01
9   2019-12-01
Name: last_pymnt_d, dtype: datetime64[ns]

In [45]:
df['last_pymnt_d'][0]

Timestamp('2019-04-01 00:00:00')

In [51]:
df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year
df[['last_pymnt_d_month', 'last_pymnt_d_year']].head(5)

,last_pymnt_d_month,last_pymnt_d_year
0,4.0,2019.0
1,12.0,2019.0
2,2.0,2019.0
3,12.0,2019.0
4,12.0,2019.0


In [53]:
df['last_pymnt_d_month'].dtype

dtype('float64')

A further task would be to convert all NaT values to "Unknown" and to run dt.year and dt.month conversion methods exclusively on non-NaT values so they result in integer values rather than float values.

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01